In [1]:
# Nos immportamos las librerias principales
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
#Descargamos las tres bases de datos.

df_Malaga = pd.read_csv("./data/Malaga.csv")
df_Valencia = pd.read_csv("./data/Valencia.csv")
df_Mallorca = pd.read_csv("./data/Mallorca.csv")

In [3]:
# Vamos a hacer una copia para trabajar sobre ella 
df_Val = df_Valencia.copy()
df_Mall = df_Mallorca.copy()
df_Mal= df_Malaga.copy()

In [4]:
# Añadimos una columna nueva con el nombre de la ciudad por si nos es útil en el futuro saber de qué ciudad se trataba.
df_Val['City'] = "Valencia"
df_Mall['City'] = "Mallorca"
df_Mal['City'] = "Malaga"


In [5]:
# Comprobamos que la haya creado.df_M
df_Mall.City.describe


<bound method NDFrame.describe of 0        Mallorca
1        Mallorca
2        Mallorca
3        Mallorca
4        Mallorca
           ...   
17210    Mallorca
17211    Mallorca
17212    Mallorca
17213    Mallorca
17214    Mallorca
Name: City, Length: 17215, dtype: object>

In [6]:
# Vamos a fusionar las tres bases de datos de las tres ciudades.
df_Dirty = pd.concat([df_Val, df_Mall,df_Mal], ignore_index=True)

In [7]:
# Veamos si se han fusionado. 
len(df_Dirty.index)
# Se han fusionado correctamente

35787

In [8]:
# Vimos que el camo 'price' tenía un "$" delante así que lo transformamos quitándole el simblo "$". 
df_Dirty['price'] = pd.to_numeric(df_Dirty['price'].str.replace('$', ''), errors='coerce')

In [9]:
# Probamos a sumarlo a ver si funciona
suma_prueba = df_Dirty['price'].sum()
print(suma_prueba)
# Parece que sí funciona. 

6523039.0


In [10]:
# Veamos los campos de la tabla fusionada. Hay muchos campos y algunos tienen muchos nulos.
df_Dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35787 entries, 0 to 35786
Data columns (total 76 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            35787 non-null  int64  
 1   listing_url                                   35787 non-null  object 
 2   scrape_id                                     35787 non-null  int64  
 3   last_scraped                                  35787 non-null  object 
 4   source                                        35787 non-null  object 
 5   name                                          35787 non-null  object 
 6   description                                   34647 non-null  object 
 7   neighborhood_overview                         15813 non-null  object 
 8   picture_url                                   35787 non-null  object 
 9   host_id                                       35787 non-null 

In [11]:
# vamos a ver la agrupación de algunos campos para ver si nos pueden interesar. 
df_Dirty.host_response_time.value_counts()
# Vemos que la mayoría de anfitriones contesta en menos de una hora.

host_response_time
within an hour        27388
within a few hours     3895
within a day           2052
a few days or more      688
Name: count, dtype: int64

In [12]:
# Por ahora nos parece útil, nos la quedamos, pero vamos a quitar algunos tipos de propiedad con los que no queremos compararnos.
df_Dirty.property_type.value_counts() 


property_type
Entire rental unit             14870
Entire villa                    5378
Entire home                     5254
Private room in rental unit     3415
Entire condo                     956
                               ...  
Holiday park                       1
Private room in camper/rv          1
Room in heritage hotel             1
Shared room in condo               1
Shared room in chalet              1
Name: count, Length: 74, dtype: int64

In [13]:
# Vamos a ver todos las posibilidades que hay, que con el value_counts no las vemos.
property_type_unique = df_Dirty['property_type'].unique()
print("Diferentes tipos de propiedad'property_type':")
print(property_type_unique)
# Hay muchos, vamos a ver otro campo para hacer el filtro.

Diferentes tipos de propiedad'property_type':
['Entire rental unit' 'Entire condo' 'Entire home'
 'Private room in rental unit' 'Entire loft' 'Entire townhouse'
 'Entire villa' 'Entire serviced apartment' 'Private room in condo'
 'Private room in home' 'Entire place' 'Tiny home' 'Boat'
 'Private room in guest suite' 'Shared room in rental unit'
 'Shared room in condo' 'Private room in townhouse'
 'Private room in bed and breakfast' 'Entire guest suite' 'Floor'
 'Private room in loft' 'Room in boutique hotel' 'Entire cottage'
 'Private room in hostel' 'Private room' 'Entire chalet' 'Room in hostel'
 'Entire vacation home' 'Casa particular' 'Private room in guesthouse'
 'Private room in casa particular' 'Private room in boat'
 'Private room in serviced apartment' 'Room in hotel'
 'Shared room in hostel' 'Room in aparthotel' 'Entire guesthouse'
 'Camper/RV' 'Private room in vacation home' 'Shared room in home'
 'Shared room in casa particular' 'Shared room in hotel' 'Houseboat'
 'Entire c

In [14]:
# Vamos a ver el room_type
df_Dirty.room_type.value_counts() 

room_type
Entire home/apt    30276
Private room        5348
Hotel room            95
Shared room           68
Name: count, dtype: int64

In [15]:
# Esta clasificación nos parece más fácil de manejar, por lo que 

# Eliminamos las filas donde 'room_type' no es un apartamento o casa entera.
room_type_keep = 'Entire home/apt'
df_Dirty = df_Dirty[df_Dirty['room_type'] == room_type_keep]

# Mostrar el DataFrame resultante
df_Dirty.room_type.value_counts() 

room_type
Entire home/apt    30276
Name: count, dtype: int64

In [16]:
# veamos la posibilidad de alquilar directamente, los resultados que da. Nos parece útil 
df_Dirty.instant_bookable.value_counts() 

instant_bookable
t    18133
f    12143
Name: count, dtype: int64

In [17]:
# veamos las respuestas positivas de los anfitriones. Nos parece útil 
df_Dirty.host_acceptance_rate.value_counts() 

host_acceptance_rate
100%    13948
99%      4440
94%      1495
98%      1471
97%       720
        ...  
15%         2
12%         2
16%         2
34%         1
5%          1
Name: count, Length: 101, dtype: int64

In [18]:
# Vamos a quitar muchas de las columnas cuya información no nos interesa. Para ello usaremos las descripcciones de la excel "Descripcción de campos"
# como son muchas haremos las eliminaremos en varias veces haciendo listas.
columnas_a_eliminar = ['listing_url', 'scrape_id','last_scraped','source','name','description','neighborhood_overview','picture_url','picture_url','host_url','host_name','host_since','host_location','host_about','host_thumbnail_url','host_picture_url','host_neighbourhood','host_total_listings_count','host_verifications','host_has_profile_pic','host_identity_verified','neighbourhood','neighbourhood_cleansed','neighbourhood_group_cleansed','bathrooms_text','amenities']
df_Dirty = df_Dirty.drop(columns=columnas_a_eliminar)

In [19]:
df_Dirty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30276 entries, 0 to 35785
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            30276 non-null  int64  
 1   host_id                                       30276 non-null  int64  
 2   host_response_time                            28946 non-null  object 
 3   host_response_rate                            28946 non-null  object 
 4   host_acceptance_rate                          29286 non-null  object 
 5   host_is_superhost                             29459 non-null  object 
 6   host_listings_count                           30276 non-null  int64  
 7   latitude                                      30276 non-null  float64
 8   longitude                                     30276 non-null  float64
 9   property_type                                 30276 non-null  obje

In [20]:
# Seguimos eliminando columnas
columnas_a_eliminar_2 = ['minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','has_availability','availability_30','availability_60','availability_90','availability_365','calendar_last_scraped','first_review','last_review','license','instant_bookable','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms']
df_Dirty_2 = df_Dirty.drop(columns=columnas_a_eliminar_2)

In [21]:
# veamos cómo ha quedado y si se nos ha pasado alguna columna por eliminar.
df_Dirty_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30276 entries, 0 to 35785
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           30276 non-null  int64  
 1   host_id                      30276 non-null  int64  
 2   host_response_time           28946 non-null  object 
 3   host_response_rate           28946 non-null  object 
 4   host_acceptance_rate         29286 non-null  object 
 5   host_is_superhost            29459 non-null  object 
 6   host_listings_count          30276 non-null  int64  
 7   latitude                     30276 non-null  float64
 8   longitude                    30276 non-null  float64
 9   property_type                30276 non-null  object 
 10  room_type                    30276 non-null  object 
 11  accommodates                 30276 non-null  int64  
 12  bathrooms                    29276 non-null  float64
 13  bedrooms             

In [22]:
# ponemos como índice 
df_Dirty_2_Index = df_Dirty_2.set_index('host_id')

In [23]:
# comprobamos que lo hemos hecho bien
indice_actual = df_Dirty_2_Index.index.name
print(f"La columna de índice actual es: {indice_actual}")

La columna de índice actual es: host_id


In [24]:
# vamos a limpiar filas. 
# Lo primero es eliminar los registros que no tienen precio porque será variable importante.
df_Dirty_2_Index.dropna(subset=['price'], inplace=True)
df_Dirty_2_Index.info()


<class 'pandas.core.frame.DataFrame'>
Index: 28559 entries, 219476 to 2634139
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           28559 non-null  int64  
 1   host_response_time           27383 non-null  object 
 2   host_response_rate           27383 non-null  object 
 3   host_acceptance_rate         27673 non-null  object 
 4   host_is_superhost            27778 non-null  object 
 5   host_listings_count          28559 non-null  int64  
 6   latitude                     28559 non-null  float64
 7   longitude                    28559 non-null  float64
 8   property_type                28559 non-null  object 
 9   room_type                    28559 non-null  object 
 10  accommodates                 28559 non-null  int64  
 11  bathrooms                    28550 non-null  float64
 12  bedrooms                     28545 non-null  float64
 13  beds          

In [25]:
# Eliminamos las filas con nulos para superhost
df_Dirty_2_Index.dropna(subset=['host_is_superhost'], inplace=True)

In [26]:
# Por último vamos a eliminar los ratings que no sean la media.
columnas_a_eliminar_3 = ['review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value']
df_Dirty_3 = df_Dirty_2_Index.drop(columns=columnas_a_eliminar_3)

In [27]:
#  Imputamos la media de bathrooms, bedrooms y beds
media_bathrooms = df_Dirty_3['bathrooms'].mean()
media_bedrooms = df_Dirty_3['bedrooms'].mean()
media_beds = df_Dirty_3['beds'].mean()
media_reviews = df_Dirty_3['reviews_per_month'].mean()
media_review_scores_rating = df_Dirty_3['review_scores_rating'].mean()

In [28]:
df_Dirty_3['bathrooms']=df_Dirty_3['bathrooms'].fillna(media_bathrooms)
df_Dirty_3['bedrooms']= df_Dirty_3['bedrooms'].fillna(media_bedrooms)
df_Dirty_3['beds']= df_Dirty_3['beds'].fillna(media_beds)
df_Dirty_3['reviews_per_month']= df_Dirty_3['reviews_per_month'].fillna(media_reviews)
df_Dirty_3['review_scores_rating']= df_Dirty_3['review_scores_rating'].fillna(media_review_scores_rating)


In [29]:
# Veamos cómo queda
df_Dirty_3.info()


<class 'pandas.core.frame.DataFrame'>
Index: 27778 entries, 219476 to 2634139
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      27778 non-null  int64  
 1   host_response_time      26644 non-null  object 
 2   host_response_rate      26644 non-null  object 
 3   host_acceptance_rate    26927 non-null  object 
 4   host_is_superhost       27778 non-null  object 
 5   host_listings_count     27778 non-null  int64  
 6   latitude                27778 non-null  float64
 7   longitude               27778 non-null  float64
 8   property_type           27778 non-null  object 
 9   room_type               27778 non-null  object 
 10  accommodates            27778 non-null  int64  
 11  bathrooms               27778 non-null  float64
 12  bedrooms                27778 non-null  float64
 13  beds                    27778 non-null  float64
 14  price                   27778 non-nu

In [30]:
# Vamos a convertir en numéricas las variables 'host_response_rate' y 'host_acceptance_rate' para luego poder sustituir por la media los nulos.

df_Dirty_3['host_response_rate'] = pd.to_numeric(df_Dirty_3['host_response_rate'].str.replace('%', ''), errors='coerce')
df_Dirty_3['host_acceptance_rate'] = pd.to_numeric(df_Dirty_3['host_acceptance_rate'].str.replace('%', ''), errors='coerce')

In [31]:
# Imprimimos las sumas para ver que ahora son numéricas.
print(df_Dirty_3['host_response_rate'].sum())
print(df_Dirty_3['host_acceptance_rate'].sum())

2578785.0
2502932.0


In [32]:
# Reemplazamos por la media los dos primeros porque son variables que hemos convertido en numéricas

df_Dirty_3['host_response_rate']=df_Dirty_3['host_response_rate'].fillna(df_Dirty_3['host_response_rate'].mean())
df_Dirty_3['host_acceptance_rate']=df_Dirty_3['host_acceptance_rate'].fillna(df_Dirty_3['host_acceptance_rate'].mean())

# y el 'host response time' por la moda, porque es una variable categórica.
df_Dirty_3.loc[df_Dirty_3.host_response_time.isna(), "host_response_time"] = df_Dirty_3.host_response_time.mode()[0]


In [33]:
# Veamos el aspecto final del dataframe.
# Veamos cómo queda
df_Dirty_3.info()


<class 'pandas.core.frame.DataFrame'>
Index: 27778 entries, 219476 to 2634139
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      27778 non-null  int64  
 1   host_response_time      27778 non-null  object 
 2   host_response_rate      27778 non-null  float64
 3   host_acceptance_rate    27778 non-null  float64
 4   host_is_superhost       27778 non-null  object 
 5   host_listings_count     27778 non-null  int64  
 6   latitude                27778 non-null  float64
 7   longitude               27778 non-null  float64
 8   property_type           27778 non-null  object 
 9   room_type               27778 non-null  object 
 10  accommodates            27778 non-null  int64  
 11  bathrooms               27778 non-null  float64
 12  bedrooms                27778 non-null  float64
 13  beds                    27778 non-null  float64
 14  price                   27778 non-nu

In [34]:
# Hacemos una copia para trabajar sobre ella:
df_final = df_Dirty_3.copy()


In [35]:
df_final.describe()

,id,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,reviews_per_month
count,2.777800e+04,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000,27778.000000
mean,4.475765e+17,96.786706,92.952501,125.590287,38.880820,0.398164,5.119807,1.827542,2.398538,3.427440,216.010044,4.110447,618.318813,31.661351,9.051336,0.913817,4.674861,1.130508
std,4.799806e+17,11.168706,16.508416,295.958485,1.261934,3.103229,2.362306,1.096204,1.380191,2.269827,154.238124,10.232044,458.277418,61.355229,14.985371,1.654606,0.360921,1.253032
min,4.815400e+04,0.000000,0.000000,1.000000,36.646012,-4.584345,1.000000,0.000000,0.000000,0.000000,10.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.010000
25%,2.871176e+07,98.000000,94.000000,2.000000,36.773368,-4.344391,4.000000,1.000000,1.000000,2.000000,115.000000,1.000000,310.250000,1.000000,0.000000,0.000000,4.630000,0.320000
50%,5.353893e+07,100.000000,99.000000,10.000000,39.474137,2.590020,4.000000,1.500000,2.000000,3.000000,165.000000,2.000000,365.000000,9.000000,3.000000,0.000000,4.680000,0.970000
75%,9.339270e+17,100.000000,100.000000,91.000000,39.708475,3.074698,6.000000,2.000000,3.000000,5.000000,263.000000,4.000000,1125.000000,33.000000,11.000000,1.000000,4.900000,1.160000
max,1.189593e+18,100.000000,100.000000,5304.000000,39.921540,3.474510,16.000000,16.500000,25.000000,50.000000,999.000000,500.000000,1125.000000,1665.000000,660.000000,78.000000,5.000000,45.000000


In [36]:
# Lo guardamos para poder usarlo con posterioridad

df_final.to_csv("./data/final.csv", index=True)
